# The pykal Workflow

**← [Back to What is PyKal?](../what_is_pykal/index.rst)**

---

Suppose we want to build a robot. If we were hobbyists, we might want a weekend project; if we were engineers, we might want a mechanical arm on the assembly line; and if we were academics, we might dream of a quadruped with a cannon-head to threaten the government for grant money.

Broadly speaking, building a robot occurs in four steps, and the ``pykal`` workflow strives to make every step as painless as possible:

As an illustrative example, let us consider building a car with cruise control. Of course, as control theorists, we lack the physical acumen to build anything more sophisticated than a paperclip, so we shall leave the construction of the car to the automotive engineers. Our realm will be control system behind the cruise control.

## Theory

We may call our car cruise control system simply the "car-cruise" system.
The block diagram for a "car-cruise" system is a simple feedback system, i.e.

![Traditional feedback system diagram](../_static/tikz/tutorial_0/feedback_system_trad.svg)

A traditional feedback control system block diagram.

where $r$ is the **setpoint** (or **reference point**),
$u$ is the **control input**,
$x$ is the (hidden) **state** of the plant,
$\hat{x}$ is the **state estimate**,
and $e$ is the **error** term. In the case of our car-cruise system, this abstraction encapsulates the following:

- setpoint generator = button on steering wheel
- controller = control algorithm and actuator
- plant = car
- observer =  sensor and estimation algorithm

To actually implement the compositions of these abstractions, we rely on the
theory of **discrete-time dynamical systems**. For the full theory as it
pertains to pykal, please click here. For a minimal working knowledge, we need
only the following.

### Discrete-time Dynamical Systems

Let $X \subset \mathbb{R}^n$ be a state manifold, let
$Y \subset \mathbb{R}^m$ be a measurement manifold, and let
$\mathcal{M} \subset \mathbb{R}^p$ be a parameter manifold in which
$X$ is embedded. A discrete-time dynamical system is the tuple
$(f, h)$, where $f$ is an evolution function
$f : \mathcal{M} \to X$ defined by $x_{k+1} = f(\mu)$, and
$h$ is an output (measurement) function
$h : \mathcal{M} \to Y$ defined by $y = h(\mu)$.

Since $X$ is embedded in $\mathcal{M}$, the state $x_k$ is
implicit in $\mu$, which is why $x_{k+1}$ is well-defined in the
evolution function (this embedding formalism is not standard among roboticists, but it is used in differential topology and will induce a particularly nice software API in the following section).

## Software

We will now cast the car-cruise system above as the composition of discrete-time
dynamical systems. First, we define a discrete-time dynamical system for each
block.

![Traditional feedback system diagram](../_static/tikz/tutorial_0/feedback_system.svg)

A traditional feedback control system block diagram.

Let $X$ be the state manifold of our car with elements
$x_k \in X$, whose state we define as velocity. Let $T$ be the
discrete-time manifold with elements $t_k \in T$.

### Setpoint Generator

Suppose our setpoint generator outputs the desired speed of our car, which we
increment or decrement with a button on our steering wheel. Thus, the setpoint
generator has a "memory" of the latest speed we gave it, and our choice of
state reflects this memory.

Let $V \subseteq X$ be our setpoint state manifold, let
$R \subseteq V$ be the setpoint output manifold, and let
$V \times U$ be the setpoint parameter manifold, where
$U = \{1, -1, 0\}$ (corresponding to increasing speed, decreasing speed,
and maintaining speed). The discrete-time dynamical system is then

$$
v_{k+1} = f(v_k, u_k) = v_k + u_k,
$$

$$
r_k = h(v_k, u_k) =
\begin{cases}
  \min(80, v_k) & \text{if } v_k \geq 80 \\
  \max(20, v_k) & \text{if } v_k \leq 0
\end{cases}
$$

In [ ]:
import numpy as np
from pykal import DynamicalSystem


def setpoint_f(vk: float, uk: float) -> float:
    return vk + uk


def setpoint_h(vk: float) -> float:
    return np.clip(vk, 0, 80.0)


setpoint_gen = DynamicalSystem(f=setpoint_f, h=setpoint_h, state_name="vk")

Notice that we have created a DynamicalSystem object that captures all of the information of our mathematical setpoint dynamical system. This information is saved to the setpoint_gen object, as we see below:

In [ ]:

print(setpoint_gen.__dict__)

The DynamicalSystem object has three attributes: two functions and a string. Even more sparse are its methods. The DynamicalSystem object has only one method, ".step()", which, true to its name, "steps" the dynamical system encapuslated by the object by passing the appropriate parameters to its functions.

We can setp once

In [ ]:
setpoint_gen.step(param_dict={"vk": vk, "uk": uk})

We can also step over time and visualize

In [ ]:

# Simulation time
T_sim = 30.0  # seconds
time_steps = np.arange(0, T_sim, dt)

vk = 0
uk = 1
for tk in time_steps:
    setpoint_gen.step(param_dict={"vk": vk, "uk": uk})

We are limited only by our creativity and progmarming ability. Here, we slam it and then chicken out and reverse it.

In [ ]:

# Simulation time
T_sim = 30.0  # seconds
time_steps = np.arange(0, T_sim, dt)

# Static control input
uk = 0
up_button_pressed = True  # programmatic latch
down_button_pressed = False

seconds_passed_since_max_speed_attained = 0
k = 0
for tk in time_steps:
    if up_button_pressed:
        vk, rk = setpoint_gen.step(return_state=True, param_dict={"vk": vk, "uk": 1})
    elif down_button_pressed:
        vk, rk = setpoint_gen.step(return_state=True, param_dict={"vk": vk, "uk": -1})


More in depth simulations will occur int he Simulation section.

### Controller

Our controller will be a discrete-time PID controller that compares the
setpoint $r_k$ to the estimated state $\hat{x}_k$ and outputs the
control input $u_k$.

The implementation can be found in [PID](../../../src/pykal/utilities/controllers/pid.py).

In [ ]:
from pykal.controllers import pid

controller = DynamicalSystem(f=pid.standard_f, h=pid.standard_h, state_name="ck")

### Plant

The plant is the car itself. For our simple cruise-control example, we model the
car as a first-order system in velocity driven by the control input $u_k$
(e.g. throttle command). Let $x_k \in X$ denote the car's velocity at
time $t_k$. A standard linear drag model with sampling time $\Delta t$
is

$$
x_{k+1} = f_p(x_k, u_k)
        = x_k + \Delta t \left(-\frac{b}{m} x_k + \frac{1}{m} u_k\right),
$$

where $m > 0$ is the (effective) mass of the car and $b \ge 0$ is a
drag coefficient.

The plant output is the (possibly noisy) measured velocity $y_k$. For now
we take a noiseless measurement model

$$
y_k = h_p(x_k) = x_k,
$$

so the plant is the discrete-time dynamical system $(f_p, h_p)$ with
parameter manifold

$$
\mathcal{M}_p = X \times U \times \mathbb{R}^2,
$$

where the last factor stores the physical parameters $(m, b)$.

In [ ]:
from numpy.random import default_rng

rng = default_rng(seed=42)


def plant_f(xk: float, uk: float, m: float, b: float, dt: float) -> float:
    return xk + dt * (-b / m * xk + uk / m)


def plant_h(xk: float) -> float:
    """Car measurement: noisy velocity observation."""
    return xk


plant = DynamicalSystem(f=plant_f, h=plant_h, state_name="xk")


### Observer

The observer reconstructs (or estimates) the hidden plant state $x_k$
from the control input $u_k$ and the plant output $y_k$. In pykal,
this role is played by a (possibly extended) Kalman filter.
[PID](../../../src/pykal/utilities/estimators/kf.py)

In [ ]:
from pykal.state_estimators import kf

observer = DynamicalSystem(f=kf.standard_f, h=kf.standard_h, state_name="xhat_P")

## Simulation

We now simulate the following scenario: we are driving along at 20 mph when, struck with the need for speed, we immediately set out cruise control to 80 mph. We also do not model noise or any other forms of data corruption; that will be covered in the next section of the tutorial.

### System Parameters

In [ ]:
# Physical parameters
m = 1500.0  # Car mass (kg)
b = 50.0  # Drag coefficient (kg/s)
dt = 0.1  # Sampling time (s)

# PID gains (hand-tuned for this system)
KP = 800.0  # Proportional gain
KI = 50.0  # Integral gain
KD = 200.0  # Derivative gain

# Kalman filter parameters (1x1 matrices for scalar system)
Q = np.array([[0.1]])  # Process noise covariance
R = np.array([[1.0]])  # Measurement noise covariance

### Initial Conditions

In [ ]:
# Initial states
ck = (0.0, 0.0, 0.0)  # Controller: (error, integral, prev_error)
xk = 20.0  # Car starts at 20 mph
xhat = xk
xhat_P = (
    np.array([[xhat]]),
    np.array([[1.0]]),
)  # Observer: (estimate as 1x1 array, covariance as 1x1 array)
vk = xhat_P[0][0, 0]  # cruise control starts at current speed estimate

# Storage for plotting
time = []
setpoints = []
true_velocities = []
estimated_velocities = []
measurements = []
control_inputs = []
errors = []

### Closed-Loop Simulation

In [ ]:
# Simulation time
T_sim = 30.0  # seconds
time_steps = np.arange(0, T_sim, dt)

# Static control input
uk = 0
up_button_pressed = True  # programmatic latch
down_button_pressed = False

seconds_passed_since_max_speed_attained = 0
k = 0
for tk in time_steps:
    if up_button_pressed:
        vk, rk = setpoint_gen.step(return_state=True, param_dict={"vk": vk, "uk": 1})
    elif down_button_pressed:
        vk, rk = setpoint_gen.step(return_state=True, param_dict={"vk": vk, "uk": -1})

    if rk == 80:
        k = k + 1
        seconds_passed_since_max_speed_attained = k * dt

    if (
        seconds_passed_since_max_speed_attained >= 5
    ):  # this reverses the logic and causes us to rapidly slow to 0
        up_button_pressed = False
        down_button_pressed = True

    # Get current state estimate from observer (extract scalar from 1x1 array)
    xhat = observer.h(xhat_P)[0, 0]

    # PID controller step
    ck, uk = controller.step(
        return_state=True,
        param_dict={
            "ck": ck,
            "rk": rk,
            "xhat_k": xhat,
            "KP": KP,
            "KI": KI,
            "KD": KD,
        },
    )

    # Plant step
    xk, yk = plant.step(
        return_state=True,
        param_dict={"xk": xk, "uk": uk, "m": m, "b": b, "dt": dt},
    )

    # Observer step (Kalman Filter) - use step() like in standard_kf.ipynb
    # Compute Jacobians as 1x1 matrices for scalar system
    Fk = np.array([[1 - dt * b / m]])  # Jacobian: ∂f/∂xk
    Hk = np.array([[1.0]])  # Measurement Jacobian (linear measurement)

    xhat_P, xhat_obs = observer.step(
        return_state=True,
        param_dict={
            "xhat_P": xhat_P,
            "yk": np.array([[yk]]),  # Convert scalar measurement to 1x1 array
            "f": plant.f,
            "f_params": {"xk": xhat, "uk": uk, "m": m, "b": b, "dt": dt},
            "h": plant.h,
            "h_params": {"xk": xhat},
            "Fk": Fk,
            "Qk": Q,
            "Hk": Hk,
            "Rk": R,
        },
    )

    # Store results
    time.append(tk)
    setpoints.append(rk)
    true_velocities.append(xk)
    estimated_velocities.append(xhat)
    measurements.append(yk)
    control_inputs.append(uk)
    errors.append(rk - xhat)

### Visualization

In [ ]:
from matplotlib import pyplot as plt

fig, axs = plt.subplots(3, 1, figsize=(12, 10))

# Plot 1: Velocity Tracking
axs[0].plot(time, setpoints, "k--", label="Setpoint (r)", linewidth=2)
axs[0].plot(time, true_velocities, "b-", label="True Velocity (x)", alpha=0.7)
axs[0].plot(time, estimated_velocities, "r-", label="Estimated Velocity (x̂)", alpha=0.7)
axs[0].scatter(
    time[::10], measurements[::10], c="gray", s=10, alpha=0.3, label="Measurements (y)"
)
axs[0].set_ylabel("Velocity (mph)", fontsize=12)
axs[0].set_title(
    "Car Cruise Control: Velocity Tracking", fontsize=14, fontweight="bold"
)
axs[0].legend(loc="upper right")
axs[0].grid(True, alpha=0.3)

# Plot 2: Control Input
axs[1].plot(time, control_inputs, "g-", label="Control Input (u)", linewidth=1.5)
axs[1].set_ylabel("Control Input (N)", fontsize=12)
axs[1].set_title("Controller Output (Throttle Command)", fontsize=14, fontweight="bold")
axs[1].legend(loc="upper right")
axs[1].grid(True, alpha=0.3)

# Plot 3: Tracking Error
axs[2].plot(time, errors, "m-", label="Tracking Error (e = r - x̂)", linewidth=1.5)
axs[2].axhline(y=0, color="k", linestyle=":", alpha=0.5)
axs[2].set_xlabel("Time (s)", fontsize=12)
axs[2].set_ylabel("Error (mph)", fontsize=12)
axs[2].set_title("Tracking Error", fontsize=14, fontweight="bold")
axs[2].legend(loc="upper right")
axs[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

We have an unnacceptable overshoot; try tuning the PID manually so that we gently rise up. Also, the button signal is sent every 0.1 seconds according to this simulation, but it can be easily modified so that it is sent every second instead (hint: multiply by 10)

---

## Navigation

**Next**: [The Curse of Hardware →](curse_of_hardware.ipynb)

**← [Back to What is PyKal?](../what_is_pykal/index.rst)**